## PersiteWrapper

This is where I develop the `PersiteWrapper` and test if we should use regularization. 
The answer is no. 
It is in `models.py` now, without regularization.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

from netam import framework, models
from netam.common import pick_device, print_parameter_count

In [3]:
train_df, val_df = framework.load_shmoof_dataframes("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv", val_nickname="59") #, sample_count=5000)

In [4]:
kmer_length = 1
site_count = 410

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, site_count=site_count)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, site_count=site_count)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Using Metal Performance Shaders
we have 44330 training examples and 4686 validation examples


In [5]:
class PersiteWrapper(nn.Module):
    """
    This wraps another model, but adds a per-site component.
    """
    def __init__(self, base_model, dataset, penalty_weight=0.0):
        super(PersiteWrapper, self).__init__()
        self.base_model = base_model
        self.site_count = dataset.max_length
        self.penalty_weight = penalty_weight
        self.log_site_rates = nn.Embedding(self.site_count, 1)

    def forward(self, encoded_parents, masks):
        base_model_rates = self.base_model(encoded_parents, masks)
        sequence_length = encoded_parents.size(1)
        positions = torch.arange(sequence_length, device=encoded_parents.device)
        log_site_rates = self.log_site_rates(positions).T
        rates = base_model_rates * torch.exp(log_site_rates)
        return rates
    
    def regularization_loss(self):
        reg_loss = torch.sum(self.log_site_rates.weight ** 2)
        reg_loss *= self.penalty_weight
        return reg_loss 

    @property
    def site_rates(self):
        # Convert site log rates to linear space
        return torch.exp(self.log_site_rates.weight).squeeze()


In [12]:
base_model_dict = {
        "cnn_sml": models.CNNModel(train_dataset, embedding_dim=6, filter_count=14, kernel_size=7, dropout_prob=0.0),
        "cnn_med": models.CNNModel(train_dataset, embedding_dim=9, filter_count=9, kernel_size=11, dropout_prob=0.1),
        "cnn_lrg": models.CNNModel(train_dataset, embedding_dim=7, filter_count=19, kernel_size=11, dropout_prob=0.3),
    }

base_model = base_model_dict["cnn_med"]
base_model.to(device)

TypeError: 'SHMoofDataset' object cannot be interpreted as an integer

In [ ]:
base_burrito = framework.SHMBurrito(train_dataset, val_dataset, base_model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-5, l2_regularization_coeff=1e-6)
print("starting training...")
losses = base_burrito.train(epochs=100)
losses.tail()

starting training...


Epoch:  53%|█████▎    | 53/100 [12:20<10:57, 13.98s/it, loss_diff=-1.747e-07, lr=6.4e-6] 


,train_loss,val_loss
49,0.061875,0.058413
50,0.061871,0.058413
51,0.061884,0.058413
52,0.061885,0.058414
53,0.061869,0.058413


In [7]:
base_model.eval() # Turn off training mode for the base model
for param in base_model.parameters():
    param.requires_grad = False

model = PersiteWrapper(base_model_dict["cnn_med"], train_dataset)
model.to(device)
burrito = framework.SHMBurrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-5, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=100)
losses.tail()

NameError: name 'base_model' is not defined

In [8]:
# Load position mutabilities
shmoof_positions = pd.read_csv('_ignore/original_shmoof/mutabilities_position.tsv', sep='\t')
# Move to 0-indexing
shmoof_positions["Position"] -= 1
cnn_residuals = torch.exp(model.log_site_rates.weight).cpu().squeeze().detach().numpy()
cnn_residuals = pd.DataFrame({'Position': np.arange(len(cnn_residuals)), 'Mutability': cnn_residuals})

# Merge dataframes
merged_positions = pd.merge(shmoof_positions, cnn_residuals, on='Position', suffixes=('_shmoof', '_cnn_residuals'))
merged_positions.sort_values(by='Position', inplace=True)

# Line plot
plt.figure(figsize=(10, 6))
plt.plot(merged_positions['Position'], merged_positions['Mutability_shmoof'], label='Shmoof')
plt.plot(merged_positions['Position'], merged_positions['Mutability_cnn_residuals'], label='CNN Residuals')
plt.xlabel('Position')
plt.ylabel('Mutability')
plt.ylim(0, 4)

plt.title('Comparison of Position Mutabilities: Shmoof vs. CNN Residuals')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'model' is not defined

In [9]:
base_model = models.CNNModel(train_dataset, embedding_dim=9, num_filters=9, kernel_size=11, dropout_prob=0.1)
model = PersiteWrapper(base_model, train_dataset)
model.to(device)
burrito = framework.SHMBurrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-5, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=100)
losses.tail()

TypeError: __init__() got an unexpected keyword argument 'num_filters'

In [10]:
regularization_coeffs = [0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
results = []
best_validation_loss = float('inf')

for coeff in regularization_coeffs:
    print(f"Training with regularization coefficient {coeff}")
    base_model = models.CNNModel(train_dataset, embedding_dim=9, num_filters=9, kernel_size=11, dropout_prob=0.1)
    model = PersiteWrapper(base_model, train_dataset, penalty_weight=coeff)
    model.to(device)
    burrito = framework.SHMBurrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, l2_regularization_coeff=1e-7)
    loss_history = burrito.train(epochs=100)
    final_training_loss = loss_history['train_loss'].iloc[-1]
    final_validation_loss = loss_history['val_loss'].iloc[-1]
    if final_validation_loss < best_validation_loss:
        best_validation_loss = final_validation_loss
        best_model = model

    results.append({
        'Regularization': coeff,
        'Final_Training_Loss': final_training_loss,
        'Final_Validation_Loss': final_validation_loss
    })

regularization_results_df = pd.DataFrame(results)
regularization_results_df

Training with regularization coefficient 0


TypeError: __init__() got an unexpected keyword argument 'num_filters'

In [11]:
plt.figure(figsize=(10, 6))
plt.plot(regularization_results_df['Regularization'], regularization_results_df['Final_Training_Loss'], label='Training Loss', marker='o')
plt.plot(regularization_results_df['Regularization'], regularization_results_df['Final_Validation_Loss'], label='Validation Loss', marker='x')

plt.xscale('log')
plt.xlabel('Regularization Coefficient')
plt.ylabel('Loss')
plt.title('Effect of L2 Regularization on Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

regularization_results_df

NameError: name 'regularization_results_df' is not defined

<Figure size 1000x600 with 0 Axes>

Conclusion: don't regularize the site rates, it doesn't help.